In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables 
Base.prepare(engine,reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [8]:
#set up inspector to get information from the SQLITE engine
myInspector = inspect(engine)
#pulls and lists column names and datatypes for future reference
columns = myInspector.get_columns("measurement")
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [9]:
#pulls and lists column names and datatypes for future reference
columns = myInspector.get_columns("station")
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [10]:
# Save references to each table
Measurement_Table = Base.classes.measurement
Station_Table = Base.classes.station

In [11]:
# Create our session (link) from Python to the DB
mySession = Session(bind=engine)

# Exploratory Climate Analysis

In [33]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database
# since session.query returns a tuple, we use [0][0] to get the value at the first entry
# and datetime.strptime to convert it to a datetime object 
# and subtract a year from it with dt.timedelta
DatePrior12Mo = dt.datetime.strptime(mySession.query(func.max(Measurement_Table.date))[0][0],'%Y-%m-%d') - dt.timedelta(days=365)
print(DatePrior12Mo)
# Perform a query to retrieve the data and precipitation scores
Last12moPrcp = mySession.query(Measurement_Table.prcp, Measurement_Table.date)
# Save the query results as a Pandas DataFrame
Last12moPrcp_df = pd.DataFrame(Last12moPrcp)
# Renamed columns to 
Last12moPrcp_df = Last12moPrcp_df.rename(columns = {"date":"Date","prcp":"Precipitation"})
# Sort the dataframe by date
Last12moPrcp_df = Last12moPrcp_df.sort_values(by="Date")
#set the index to the date column
Last12moPrcp_df = Last12moPrcp_df.set_index("Date")

# Use Pandas Plotting with Matplotlib to plot the data
Last12moPrcp_df.plot(kind='bar')
plt.ylabel("Inches")

2016-08-23 00:00:00


AttributeError: 'Line2D' object has no property 'rotation'

In [13]:
# Use Pandas to calcualte the summary statistics for the precipitation data
Last12moPrcp_df.describe()

,Precipitation
count,18103.000000
mean,0.160644
std,0.468746
min,0.000000
25%,0.000000
50%,0.010000
75%,0.110000
max,11.530000


In [26]:
# Design a query to show how many stations are available in this dataset?
QueryCount_tu = mySession.query(func.count(Station_Table.station))
QueryCount = QueryCount_tu[0][0]
print(QueryCount)

9


In [29]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
QueryCount_1 = mySession.query(Station_Table.station).count()
#QueryCount = QueryCount_tu[0][0]
print(QueryCount_1)

9


In [16]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [17]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [18]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

NameError: name 'session' is not defined

In [19]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [20]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [21]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [22]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

NameError: name 'Measurement' is not defined

In [23]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [24]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [25]:
# Plot the daily normals as an area plot with `stacked=False`
